In [1]:
import os
import numpy as np
import pickle as pc
import tensorflow as tf
from utils import *

#from seq2seq import *
from LSTM import *

In [2]:
LBNL_DATA_DIR = 'data/LBNL Building 74/lbnlb74electricity.xlsx'
horizon_size = 4
best_loss = 100000000.0
lr = 0.0001
batch_size = 16
max_patience = 7
max_epochs = 100
checkpoint_dir = 'lstm2_checkpoint'
LOG_INTERVAL = 200
keep_rate = 0.5
hidden_unit = 16
num_layer = 1

debug = False

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [3]:
def train():
    # data load
    time, elec = load_data(LBNL_DATA_DIR)
    
    # modify
    elec = missing_value(elec)
    print(elec[40226])
    
    elec = std_normalize(elec)
    # split data
    train, valid, test = split_dataset(elec)
    
    # build dataset
    train_observe, train_pred = lstm_build_dataset(train, horizon_size)
    val_observe, val_pred = lstm_build_dataset(valid, horizon_size)
    print("Build Dataset Finished")
    print("[Train] enc {}\tdec {}".format(len(train_observe), len(train_pred)))
    print("[Dev] enc {}\tdec {}".format(len(val_observe), len(val_pred)))
    
    # model
    lstm = LSTM(hidden_unit, keep_rate)
    
    # optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    
    # loss & metric obj
    loss_obj = tf.keras.losses.MeanSquaredError()
    rmse_obj = tf.keras.metrics.RootMeanSquaredError()
    val_rmse_obj = tf.keras.metrics.RootMeanSquaredError()
    
    # Earlystopping
    earlystopping = EarlyStopping(best_loss, max_patience)
    
    # ckpt dir
    ckpt_dir = os.path.join(checkpoint_dir, 'lr-{}_hidden-{}_hr-{}'.format(lr, hidden_unit, horizon_size))
    best_ckpt_dir = os.path.join(checkpoint_dir, 'best_lr-{}_hidden-{}_hr-{}'.format(lr, hidden_unit, horizon_size))
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)
    if not os.path.exists(best_ckpt_dir):
        os.makedirs(best_ckpt_dir)
        
    checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                     lstm=lstm)
    
    num_batches_per_epoch = (len(train_observe)-1) // batch_size + 1
    print("num_batches_per_epoch: {}".format(num_batches_per_epoch))
    
    for epoch in range(max_epochs):
        
        
        
        train_batches = batch_iter(train_observe, train_pred, batch_size)        
        
        train_loss = list()
        val_loss = list()
        
        for batch_idx, (batch_x, batch_y) in enumerate(train_batches):
            
            with tf.GradientTape() as tape:
                
                logits = lstm(batch_x, training=True)
                
                loss = loss_obj(batch_y, logits)
                    
                trainable_variables = lstm.trainable_variables
            
            gradients = tape.gradient(loss, trainable_variables)
            optimizer.apply_gradients(zip(gradients, trainable_variables))
        
            rmse_obj.update_state(batch_y, logits)
            train_loss.append(np.mean(loss))

            if (batch_idx+1) % LOG_INTERVAL == 0:
                print("[epoch {} | step {}/{}] loss: {:.4f} (Avg. {:.4f}) RMSE: {:.4f}".format(epoch+1, 
                                                                                               batch_idx+1, 
                                                                                               num_batches_per_epoch, 
                                                                                               np.mean(loss), 
                                                                                               np.mean(train_loss), 
                                                                                               rmse_obj.result().numpy()))
        val_batches = batch_iter(val_observe, val_pred, batch_size)
        
        for val_batch_idx, (val_batch_x, val_batch_y) in enumerate(val_batches):
            
            val_logits = lstm(val_batch_x, training=False)
                   
            loss = loss_obj(val_batch_y, val_logits)

            val_rmse_obj.update_state(val_batch_y, val_logits)
            val_loss.append(np.mean(loss))
            
        print("[epoch {}] loss: {:.4f} RMSE: {:.4f}".format(epoch + 1, np.mean(val_loss), val_rmse_obj.result().numpy()))

        # applying earlystopping
        early = earlystopping.update(np.mean(val_loss), epoch)
        if early == 'update':
            ckpt_prefix = os.path.join(best_ckpt_dir, 'best_ckpt_{}'.format(epoch+1))            
            checkpoint.save(file_prefix = ckpt_prefix)
            print("[epoch {} patience {} max_patience {} best_loss {}]\tModel best performance!".format(epoch+1, earlystopping.patience, earlystopping.max_patience, earlystopping.best_loss))
        elif early == 'patience':
            ckpt_prefix = os.path.join(ckpt_dir, 'ckpt_{}'.format(epoch+1))                    
            checkpoint.save(file_prefix = ckpt_prefix)
            print("[epoch {} patience {} max_patience {} best_loss {}]\tModel is saved".format(epoch+1, earlystopping.patience, earlystopping.max_patience, earlystopping.best_loss))
        else:
            print("[epoch {} best_epoch {} patience {} max_patience {} best_loss {}]\tTraining process is finished".format(epoch+1, earlystopping.best_epoch+1, earlystopping.patience, earlystopping.max_patience, earlystopping.best_loss))
            break
        rmse_obj.reset_states()
        val_rmse_obj.reset_states()

In [4]:
if __name__ == '__main__':
    train()

/usr/local/lib/python3.6/dist-packages/openpyxl/worksheet/_reader.py:292: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Size of time, elec: 46111	46111
43.899
Mean: 31.535117450933615	Std: 8.103161225783838
Size of train, valid, test: 27973	6994	11144
Build Dataset Finished
[Train] enc 27969	dec 27969
[Dev] enc 6990	dec 6990


W1208 12:27:15.395750 140153360496448 base_layer.py:1772] Layer lstm is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



num_batches_per_epoch: 1749
[epoch 1 | step 200/1749] loss: 0.1229 (Avg. 0.3425) RMSE: 0.5852
[epoch 1 | step 400/1749] loss: 0.0524 (Avg. 0.2658) RMSE: 0.5155
[epoch 1 | step 600/1749] loss: 0.0438 (Avg. 0.2384) RMSE: 0.4882
[epoch 1 | step 800/1749] loss: 1.8308 (Avg. 0.2251) RMSE: 0.4745
[epoch 1 | step 1000/1749] loss: 0.0660 (Avg. 0.2117) RMSE: 0.4601
[epoch 1 | step 1200/1749] loss: 0.0590 (Avg. 0.1989) RMSE: 0.4460


KeyboardInterrupt: 